# SpeakUp2.0: Video Clipping and Audio-Video Alignment

### Info Documents 
Location Repository
Github Repository 
Jupyter Notebook


### Requirements
Please install the necessary packages in requirements.txt using pip install -r requirements.txt

## 0. Importing Relevant Packages 

In [1]:
import os             # Importing the os module which provides functions for interacting with the operating system
import pyxdf          # Importing pyxdf, a Python library for reading XDF files
import glob           # Importing the glob module which helps in finding files/directories with specific patterns
import pandas as pd   # Importing pandas library (abbreviated as pd), which is used for data manipulation and analysis
import numpy as np    # Importing numpy library (abbreviated as np), which is used for numerical computations
import wave           # Importing wave module for reading and writing WAV files (usually audio files) 
import struct         # Importing struct module which provides functions to convert between Python values and C structs
import math           # Importing math module which provides mathematical functions
import random         # Importing random module for generating random numbers
from scipy.io import wavfile  # Importing wavfile module from scipy.io (a library built on numpy), for reading and writing WAV files
import noisereduce as nr      # Importing noisereduce module for noise reduction in audio signals
import json            # Importing json module for working with JSON data
import cv2            # Importing OpenCV library for computer vision tasks
from moviepy.editor import (                # Importing various classes and functions from moviepy.editor module
                            VideoFileClip,  # Class for working with video files
                            AudioFileClip,  # Class for working with audio files
                            CompositeAudioClip)  # Class for composing audio clip
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip  # video  clipping fucntion 
from moviepy.video.io.VideoFileClip import VideoFileClip          # alternative video clipping function
import matplotlib.pyplot as plt                                   # Importing pyplot library to create figures and plot data 
from matplotlib.widgets import Slider  
import tkinter                                                    # GUI toolkit to open and save files
from tkinter import filedialog                                    # GUI toolkit to open and save files
import subprocess 
from tqdm.notebook import tqdm
import re                                                         # Importing re module for working with regular expressions


print("Everything was imported succesfully") #as terminal

c:\Users\marti\git\Multimodal\multimodal_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Everything was imported succesfully


## 1. Establish Relevant Paths, Variabls & Functions 

In [16]:
# ------------ PATHS -----------------------------------------------------
input_video_folder = r'\\fileserver.dccn.nl\project\3025011.01\pilots\pilot_data_davide'
#input_file_folder   = '../1_XDF_PROCESSING/data_processed/lateststart_earliestend'    # this folder contains the csv files extracted from the XDF files (data_processed)
#input_file_folder =   '../1_XDF_PROCESSING/data_processed_temp/manual_startend'
input_file_folder   = '../1_XDF_PROCESSING/data_processed_final/marker_3pairs'    # this folder contains the csv files extracted from the XDF files (data_processed)
output_video_folder = './video_clipped/marker_3pairs/'     # this folder will contain the clipped videos
output_audiovideo_folder = './audiovideo_merged/marker_3pairs'     # this folder will contain the audio files extracted from the videos
#output_overlay_folder = './audiovideo_overlay/lateststart_earliestend' 

print("Input video folder =", os.path.abspath(input_video_folder))
print("Input file folder =", os.path.abspath(input_file_folder))
print("Output_video folder =", os.path.abspath(output_video_folder))
print("Output_audiovideo folder =", os.path.abspath(output_audiovideo_folder))


# ------------ VARIABLES -----------------------------------------------------
# Dictionary to map file extensions to codecs
extension_to_codec = {
    '.mp4': 'libx264',
    '.avi': 'XVID',
    '.mov': 'libx264',
    '.mkv': 'libx264',
    '.flv': 'flv',
    # Add more mappings as needed
                    }


# AUDIO-VIDEO MATCHING: Regular expression pattern to match the participant's video file name. Change as needed. 
audio_video_pattern_matching = re.compile(r'pilot_(\d+)_d(\d+)')
# pilot_01_d2_speech_lsl_Mic_latest_start_earliest_end_denoised.wav
# pilot_05_d2_speech_output_comp
# pilot_01_d2_speech_lsl_Video_latest_start_earliest_end_denoised.wav

"""In our case the files are named as # pilot_xx_dx_speech_lsl_Mic_latest_start_earliest_end_denoised.wav and # pilot_xx_dx_speech_output_comp
        where x are changing expression inside this regular pattern. 
            pilot_: Matches the static string "pilot_".
            (\d+): Captures one or more digits (xx in your case).
            _d(\d+): Captures the dynamic d part followed by digits (dx).
            _speech: Matches the static string _speech.
    The regular expression is used to match the file names between audios and videos audiio audio-video alignment"""



# Dictionary to map file extensions to FourCC codecs
extension_to_codec = {
    '.mp4': 'H264',  # 'H264' is a valid FourCC code for H.264
    '.avi': 'XVID',  # 'XVID' is a common FourCC for AVI
    '.mov': 'H264',  # Again, use 'H264' or 'MP4V'
    '.mkv': 'H264',  # Use 'H264' for MKV files too
    '.flv': 'FLV1',  # 'FLV1' is a valid codec for FLV
    # Add more mappings as needed
}


# ------------ FUNCTIONS -----------------------------------------------------
# VIDEO: Creating a function named frame_to_time to convert frame number to time format 
def frame_to_time(frame, fps):
    """
    frame_to_time converts a given frame number to a time format (HH:MM:SS.SS) based on the frames per second (fps).
    Arguments:
        frame (int): The frame number to be converted.
        fps (float): The frames per second of the video.
    Returns:
        str: The time format as a string in the format "HH:MM:SS.SS".
    """
    seconds = frame / fps
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:.2f}"

print("Function \"frame_to_time\" created sucesfully") 



# Function to extract specified events (with correspodning LSL times) from XDF stream (useful for plotting)
def get_events(stream, event_names):
    """
    Extracts events and corresponding LSL times from the given stream that match any of the event_names.

    Parameters:
    stream (dict): The stream containing time stamps and event data.
    event_names (list of str): List of event name substrings to look for in the events.

    Returns:
    np.array: An array where each row contains a timestamp and the full event name.
    """
    events = []  # Initialize an empty list to store matching events

    # Check if the stream type is "Markers"
    if stream['info']['type'][0] != "Markers":
        raise ValueError(f"ERROR: The stream provided ({stream['info']['name'][0]}) is not a Marker stream")

    # Iterate over the time stamps and corresponding events in the stream
    for timestamp, event in zip(stream['time_stamps'], stream['time_series']):
        # Check if any of the specified event names are in the current event
        for name in event_names:
            if name in event[0]:
                # If a match is found, append the timestamp and full event name to the list
                events.append([timestamp, event[0]])

    # Convert the list of events to a NumPy array and return it
    return np.array(events)

print("Function \"get_events\" created sucesfully") 

Input video folder = \\fileserver.dccn.nl\project\3025011.01\pilots\pilot_data_davide
Input file folder = f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs
Output_video folder = f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs
Output_audiovideo folder = f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\audiovideo_merged\marker_3pairs
Function "frame_to_time" created sucesfully
Function "get_events" created sucesfully


### 2. Identify Relevant CVS files, Audio Files and Videos by navigating through Input Folders 

In [4]:
## 1. FIND LSL VIDEO FILES (csv files) IN THE INPUT FOLDER
LSL_video_list = []  # Initialize an empty list to store paths of video files (csv LSL data)
try:
    if os.path.exists(input_file_folder):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(os.path.abspath(input_file_folder)):  # Walk through the directory tree
          
            for file in files:
                
                if "speech" in file and 'Video' in file and file.endswith(".csv"):  # Check that the words "speech" and "Video" are in the file name and that it is a CSV file
                    
                    LSL_video_list.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if LSL_video_list:  # Check if any files were found
            print(f'We have identified the following LSL video files: {LSL_video_list}')
        
        else:  # If no XDF files were found
            print('No LSL video files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_file_folder} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")
    
    
    
    
## 2. FIND AUDIO FILES (.wav files) IN THE INPUT FILE FOLDER
audio_list = []  # Initialize an empty list to store paths of video files (csv LSL data)
try:
    if os.path.exists(input_video_folder):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(os.path.abspath(input_file_folder)):  # Walk through the directory tree
          
            for file in files:
                
                if "speech" in file and 'Mic' in file and 'denoised' in file and file.endswith(".wav"):  # Check that the words "speech" and "Mic" and "denoised" are in the file name and that it is a wav file
                    
                    audio_list.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if audio_list:  # Check if any files were found
            print(f'We have identified the following Audio files: {audio_list}')
        
        else:  # If no  files were found
            print('No audio files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_file_folder} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")
    
    
    
## 2. FIND RAW VIDEO FILES IN THE INPUT VIDEO FOLDER
raw_video_list = []  # Initialize an empty list to store paths of video files (csv LSL data)
try:
    if os.path.exists(input_video_folder):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(os.path.abspath(input_video_folder)):  # Walk through the directory tree
          
            for file in files:
                
                if "speech" in file and 'output_compr' in file and file.endswith(".avi"):  # Check that the words "speech" and "output_compr" are in the file name and that it is a avi file
                    
                    raw_video_list.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if raw_video_list:  # Check if any files were found
            print(f'We have identified the following Video files: {raw_video_list}')
        
        else:  # If no files were found
            print('No video files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_video_folder} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")

We have identified the following LSL video files: ['f:\\SpeakUp-2.0\\2_PREPROCESSING\\1_XDF_PROCESSING\\data_processed_final\\marker_3pairs\\pilot_01\\day2\\pilot_01_d2_speech_lsl_Video_Speech_Start_SUD_1_Start.csv', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\1_XDF_PROCESSING\\data_processed_final\\marker_3pairs\\pilot_01\\day2\\pilot_01_d2_speech_lsl_Video_SUD_1_End_SUD_2_Start.csv', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\1_XDF_PROCESSING\\data_processed_final\\marker_3pairs\\pilot_01\\day2\\pilot_01_d2_speech_lsl_Video_SUD_2_End_SUD_3_Start.csv', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\1_XDF_PROCESSING\\data_processed_final\\marker_3pairs\\pilot_01\\day3\\pilot_01_d3_speech_lsl_Video_Speech_Start_SUD_1_Start.csv', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\1_XDF_PROCESSING\\data_processed_final\\marker_3pairs\\pilot_01\\day3\\pilot_01_d3_speech_lsl_Video_SUD_1_End_SUD_2_Start.csv', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\1_XDF_PROCESSING\\data_processed_final\\marker_3pairs\\pilot_01\\day3\\pilot_01_d3_speech_lsl

## 3. Clipping Videos Based on .CSV frames and LSL times

In [5]:
##1. LOAD LSL VIDEO FILES  
# Loop through the files 
for file in LSL_video_list: 
    
    print(f'Processing {file}')
    
    file_path = os.path.join(os.path.abspath(input_file_folder), file)  # Create the file path for the CSV file
    
    file_data = pd.read_csv(file_path)
        
    # Find the start and end frame numbers in the CSV file 
    start_frame = file_data.iloc[0, 1]    # Get the first frame (first row, index 0) in the second coluimn (index 1)
    end_frame   = file_data.iloc[-1, 1]   # Get the last frame (last row, index -1) in the second coluimn (index 1)
    
    fnam = os.path.basename(file)[:-4]     # Extract the file name from the path and assings it to fnam, whilst removing the '.csv' extension (i.e., the last 4 characters in the string)
    
    participant_num = '_'.join(fnam.split('_')[0:2])  # Extract the participant number from the file name by splitting the string at the underscores and selecting the first two elements
    
    day_num = fnam.split('_')[2]  # Extract the day number from the file name by splitting the string at the underscores and selecting the third element

    condition = '_'.join(fnam.split('_')[6:])  # Extract the condition from the file name by splitting the string at the underscores and selecting the elements from the 7th to the last element
    
    video_extension = '.avi'
    
    # Create the output subfolder for the clipped videos
    subfolder_path = os.path.join(os.path.abspath(output_video_folder), participant_num, day_num, f'{participant_num}_{day_num}_{condition}{video_extension}')  # Create the output subfolder for the clipped video with the correct extension
    print(f'Subfolder path: {subfolder_path}')
    # Make sure the output folder exists, if not create it
    if not os.path.exists(os.path.dirname(subfolder_path)):
        os.makedirs(os.path.dirname(subfolder_path), exist_ok=True)
        
    
    # Extract the identifier from the audio file name that will be used to match the video file
    LSL_video_match = audio_video_pattern_matching.search(fnam)
    if not LSL_video_match :
        # print(f'No matching for video: {fnam}')
        continue  # Skip current iteration if the pattern is not found
    
    LSL_video_identifier = LSL_video_match.group(0)  # Extract the matched portion of the string from the regex result.
    #print(f'Video identifier: {LSL_video_identifier}')  # Print the matched identifier for audio

        
    # Find the frame rate of the LSL frames (i.e., the number of frames per second)
    LSL_frame_rate = (end_frame - start_frame) / (file_data.iloc[-1, 0] - file_data.iloc[0, 0])  # Calculate the frame rate by dividing the number of frames by the time difference between the first and last frame
        
        
    # 2. LOAD CORRESPDONDING RAW VIDELoad
    for video in raw_video_list :  # Loop through the raw video files to find the corresponding video file
        
        print(f'Now Loading the Video :  {os.path.basename(video)}')
        
        fnam = os.path.basename(video)[:-4]     # Extract the file name from the path and assings it to fnam, whilst removing the '.csv' extension (i.e., the last 4 characters in the string)
        
        # Extract the identifier from the audio file name that will be used to match the video file
        raw_video_match = audio_video_pattern_matching.search(fnam)
        if not raw_video_match :
            # print(f'No matching for video: {fnam}')
            continue  # Skip current iteration if the pattern is not found
        
        raw_video_identifier = raw_video_match.group(0)  # Extract the matched portion of the string from the regex result.
        #print(f'Video identifier: {LSL_video_identifier}')  # Print the matched identifier for audio
        
        
        # Check that the LSL and raw video identifiers match
        if LSL_video_identifier == raw_video_identifier: 
            
            print(f'Found matching raw video: {os.path.basename(video)} for LSL video: {os.path.basename(file)}')
                
            video_path = os.path.join(os.path.abspath(input_video_folder), video)
             
            capture = cv2.VideoCapture(video_path) # Load the video using OpenCV
                
            # Extract the relevant metadata from the video
            video_frame_width  = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))  
            video_frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))  
            video_frame_rate   = capture.get(cv2.CAP_PROP_FPS)
            video_tot_frames   = int(capture.get(cv2.CAP_PROP_FRAME_COUNT)) 
            
            # Check video extension format and select the appropriate codec in the dictionary
            video_extension = os.path.splitext(video_path)[1].lower()  # Extract the file extension from the video file path, ensuring it's in lowercase (e.g., '.mp4'), to enable case-insensitive matching in the dictionary
            if video_extension in extension_to_codec:
                codec = extension_to_codec[video_extension]
            else:
                raise ValueError(f"ERROR: The video extension {video_extension} is not supported")
            
            
           # -------------------------------------------------------------------------------------------- 
            
            # ## 3. CLIP THE VIDEO USING FFMPEG (FASTER, BASED ON TIMES (converted from frame numbers))
           
            # # Converting the LSL frames to the video time format to find start_cut and end_cut for the video
            # start_cut_time = frame_to_time(start_frame, LSL_frame_rate)
            # end_cut_time = frame_to_time(end_frame, LSL_frame_rate)

            # print("start_cut_time" + str(start_cut_time))
            # print("end_cut_time" + str(end_cut_time))

            # print('Now cutting the video...')
            
            # # Construct output file path with the same extension
            # subfolder_path = os.path.join(os.path.abspath(output_video_folder), participant_num, day_num, f'{participant_num}_{day_num}_speech_video_clipped{video_extension}')  # Create the output subfolder for the clipped video with the correct extension
            # # Make sure the output folder exists, if not create it
            # if not os.path.exists(os.path.dirname(subfolder_path)):
            #     os.makedirs(os.path.dirname(subfolder_path), exist_ok=True)
            
            
            # # Use ffmpeg to cut the video
            # ffmpeg_command = [
            #     'ffmpeg',
            #     '-y',                  # Add -y flag to overwrite any existing files with the same name
            #     '-i', video_path,
            #     '-ss', start_cut_time,  # start time
            #     '-to', end_cut_time,    # end time
            #     '-c', 'copy',           # copy codec (no re-encoding)
            #     subfolder_path]
            

            # # Execute the command
            # subprocess.run(ffmpeg_command, check=True)

            # print(f'Video saved as {subfolder_path}')

            # # --------------------------------------------------------------------------------------------
            
            ## 3a CLIP THE VIDEO USING MOVIEPY (SLOWER, BASED ON TIME)  
            # Assign video extension to Four Character Code (fourcc) for the codec
            fourcc = cv2.VideoWriter_fourcc(*codec)
            
            # output_video_folder ='./video_clipped_temp'  
        
                
            # find the codec based on the video extension 
            if video_extension in extension_to_codec:
                codec = extension_to_codec[video_extension]
            else:
                raise ValueError(f"ERROR: The video extension {video_extension} is not supported")
            
            # assign code to Four Character Code (fourcc) for the codec 
            fourcc = cv2.VideoWriter_fourcc(*codec)
                
            # Initialize the VideoWriter object to write frames to a new video file based on the fourcc codec, LSL frame rate, video frame size, and subfolder path
            out = cv2.VideoWriter(subfolder_path, fourcc, LSL_frame_rate, (video_frame_width, video_frame_height))  
            
            # Write the selected frames to the a new clipped video 
            capture.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # Set the video capture object to the start frame
            with tqdm(total= end_frame - start_frame + 1,   desc="Rewriting Video Progress", leave=False, ncols=100) as pbar:    # Create a progress bar for the video frames

                frame_count = start_frame  # Initialize the frame count to the start frame
            
                while capture.isOpened() and frame_count <= end_frame:  # Loop through the video frames from the start frame until the end frame is reached

                    try: 
                        ret, frame = capture.read()   # Read the next frame from the video
                        if ret:   # If the frame is read correctly
                            frame_count += 1
                            pbar.update(1)
                            out.write(frame)  # Write the frame to the new video
                        else:
                            break
                        
                    except Exception as e:  # Catch any errors that occur during the frame processing
                        print(f"An error occurred at frame {frame_count}: {e}")
                        break  # Stop processing if an error occurs
                    
                
            # Release the video capture and video writer objects
            capture.release()
            out.release()
        
        print(f'Clipped video saved to: {subfolder_path}')
            
        print("\n")
        
    else: # If no matching raw video is found
        print(f'No matching raw video found for LSL video: {os.path.basename(file)}')
                
                
print("All videos have been clipped successfully. Look into your folder " + output_video_folder) #as terminal            # Create the output subfolder for the clipped videos 

Processing f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs\pilot_01\day2\pilot_01_d2_speech_lsl_Video_Speech_Start_SUD_1_Start.csv
Subfolder path: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start.avi
Now Loading the Video :  pilot_01_d2_speech_output_compr.avi
Found matching raw video: pilot_01_d2_speech_output_compr.avi for LSL video: pilot_01_d2_speech_lsl_Video_Speech_Start_SUD_1_Start.csv


Rewriting Video Progress:   0%|                                          | 0/7237.0 [00:00<?, ?it/s]

Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start.avi


Now Loading the Video :  pilot_01_d3_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start.avi


Now Loading the Video :  pilot_02_d2_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start.avi


Now Loading the Video :  pilot_02_d3_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start.avi


Now Loading the Video :  pilot_03_d2_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Sp

Rewriting Video Progress:   0%|                                          | 0/6814.0 [00:00<?, ?it/s]

Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_SUD_1_End_SUD_2_Start.avi


Now Loading the Video :  pilot_01_d3_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_SUD_1_End_SUD_2_Start.avi


Now Loading the Video :  pilot_02_d2_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_SUD_1_End_SUD_2_Start.avi


Now Loading the Video :  pilot_02_d3_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_SUD_1_End_SUD_2_Start.avi


Now Loading the Video :  pilot_03_d2_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_SUD_1_End_SUD_

...

Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_Speech_Start_SUD_1_Start.avi


No matching raw video found for LSL video: pilot_06_d2_speech_lsl_Video_Speech_Start_SUD_1_Start.csv
Processing f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs\pilot_06\day2\pilot_06_d2_speech_lsl_Video_SUD_1_End_SUD_2_Start.csv
Subfolder path: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_1_End_SUD_2_Start.avi
Now Loading the Video :  pilot_01_d2_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_1_End_SUD_2_Start.avi


Now Loading the Video :  pilot_01_d3_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_1_End_SUD_2_Start.avi


Now L

Rewriting Video Progress:   0%|                                          | 0/6904.0 [00:00<?, ?it/s]

Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_1_End_SUD_2_Start.avi


No matching raw video found for LSL video: pilot_06_d2_speech_lsl_Video_SUD_1_End_SUD_2_Start.csv
Processing f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs\pilot_06\day2\pilot_06_d2_speech_lsl_Video_SUD_2_End_SUD_3_Start.csv
Subfolder path: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_2_End_SUD_3_Start.avi
Now Loading the Video :  pilot_01_d2_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_2_End_SUD_3_Start.avi


Now Loading the Video :  pilot_01_d3_speech_output_compr.avi
Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_2_End_SUD_3_Start.avi


Now Loading

Rewriting Video Progress:   0%|                                          | 0/6890.0 [00:00<?, ?it/s]

Clipped video saved to: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_06\d2\pilot_06_d2_SUD_2_End_SUD_3_Start.avi


No matching raw video found for LSL video: pilot_06_d2_speech_lsl_Video_SUD_2_End_SUD_3_Start.csv
All videos have been clipped successfully. Look into your folder ./video_clipped/marker_3pairs/


## 3. Merging (Clipped) Vidoes with (Clipped) Audios 

In [11]:
## FIRST RE-CREATE A LIST OF THE CLIPPED VIDEO FILES 
input_video_folder = './video_clipped/marker_3pairs'     # this folder contains the clipped videos

condition_1 = 'Speech_Start_SUD_1_Start'  # The condition to search for in the file name

condition_2 = 'SUD_1_End_SUD_2_Start'  # The condition to search for in the file name

condition_3 = 'SUD_2_End_SUD_3_Start'  # The condition to search for in the file name

clipped_video_list = []  # Initialize an empty list to store paths of video files (csv LSL data)
try:
    if os.path.exists(input_video_folder):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find video files
        for root, dirs, files in os.walk(os.path.abspath(input_video_folder)):  # Walk through the directory tree
          
            for file in files:
                
                if condition_1 in file and file.endswith(".avi"):  # Check that the words "speech" and "output_compr" are in the file name and that it is a avi file
                    
                    clipped_video_list.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if clipped_video_list :  # Check if any files were found
            print(f'We have identified the following Video files: {clipped_video_list}')
        
        else:  # If no files were found
            print('No video files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_video_folder} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")



## Also RE-CREATE A LIST OF THE AUDIO FILES TO match only 1 condition at at time 
## 2. FIND AUDIO FILES (.wav files) IN THE INPUT FILE FOLDER
audio_list = []  # Initialize an empty list to store paths of video files (csv LSL data)
try:
    if os.path.exists(input_video_folder):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(os.path.abspath(input_file_folder)):  # Walk through the directory tree
          
            for file in files:
                
                if condition_1 in file and 'Mic' in file and 'denoised' in file and file.endswith(".wav"):  # Check that the words "speech" and "Mic" and "denoised" are in the file name and that it is a wav file
                    
                    audio_list.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if audio_list:  # Check if any files were found
            print(f'We have identified the following Audio files: {audio_list}')
        
        else:  # If no  files were found
            print('No audio files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_file_folder} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")


We have identified the following Video files: ['f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_01\\d2\\pilot_01_d2_Speech_Start_SUD_1_Start.avi', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_01\\d3\\pilot_01_d3_Speech_Start_SUD_1_Start.avi', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_02\\d2\\pilot_02_d2_Speech_Start_SUD_1_Start.avi', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_02\\d3\\pilot_02_d3_Speech_Start_SUD_1_Start.avi', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_04\\d2\\pilot_04_d2_Speech_Start_SUD_1_Start.avi', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_05\\d2\\pilot_05_d2_Speech_Start_SUD_1_Start.avi', 'f:\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\video_clipped\\marker_3pairs\\pilot_06\\d2\\pi

In [17]:
# THEN WE USE THESE CLIPPED VIDEOS AND ALIGNM THEM TO THE CORREPSONDING AUDIO 
# LOADING AUDIOS: Loop over files in the input_file_folder to extract the relevant audio files


for audio in audio_list:    
        
    print(f'Processing the audio file:  {audio}')
    
    audio_path = os.path.join(os.path.abspath(input_file_folder), audio)
    
    fnam = os.path.basename(audio)[:-4]  
    
    participant_num = '_'.join(fnam.split('_')[0:2])  # Extract the participant number from the file name by splitting the string at the underscores and selecting the first two elements
    print(f'Participant number: {participant_num}')  # Print the participant number
    
    day_num = fnam.split('_')[2]  # Extract the day number from the file name by splitting the string at the underscores and selecting the third element
    print(f'Day number: {day_num}')  # Print the day number

    condition = '_'.join(fnam.split('_')[6:])  # Extract the condition from the file name by splitting the string at the underscores and selecting the elements from the 7th to the last element
    
    # Extract the identifier from the audio file name that will be used to match the video file
    audio_match = audio_video_pattern_matching.search(fnam)
    if not audio_match:
        print(f'No matching for audio: {audio}')
        continue  # Skip current iteration if the pattern is not found
    
    audio_identifier = audio_match.group(0)  # Extract the matched portion of the string from the regex result.
    #print(f'Audio identifier: {audio_identifier}')  # Print the matched identifier for audio


    # LOADING VIDEOS: Loop over video files to select the corresponding video file
    for video in clipped_video_list:
        
        fnam = os.path.basename(video)[:-4]  
        
        # Extract the identifier from the video file name that will be used to match the audio file
        video_match = audio_video_pattern_matching.search(fnam)
        if not video_match:
            print(f'No matching for video: {video}')
            continue  # Skip current iteration if the pattern is not found
        
        video_identifier = video_match.group(0)  # Extract the matched portion of the string from the regex result.
        #print(f'Video identifier: {video_identifier}')  # Print the matched identifier for video

    
        # Check if the identifiers from the audio and video files match
        if audio_identifier == video_identifier: 
            
            video_path = os.path.join(os.path.abspath(input_video_folder), video)
            video_extension = os.path.splitext(video_path)[1].lower()  # Ensure correct video extension
            
            print(f'Found matching video: {video} for audio: {audio}')
            
            # Create output folder if it doesn't exist
            os.makedirs(output_audiovideo_folder, exist_ok=True)
            
            # ALIGNMENT: Combining Audio and Video using ffmpeg 
            subfolder_path = os.path.join(os.path.abspath(output_audiovideo_folder), participant_num, day_num, f'{video_identifier}_{condition}_audiovideo_merged.{video_extension[1:]}')
            
            #make sure the output folder exists, if not create it
            if not os.path.exists(os.path.dirname(subfolder_path)):
                os.makedirs(os.path.dirname(subfolder_path), exist_ok=True)
                
            
            # Construct the ffmpeg command to combine the audio and video files
            ffmpeg_command = [
                'ffmpeg',              # Call the ffmpeg tool
                '-y',                  # Overwrite the output file without asking
                '-i', video_path,      # Specify the input video file
                '-i', audio_path,      # Specify the input audio file
                '-c:v', 'copy',        # Copy the video stream without re-encoding
                '-c:a', 'aac',         # Re-encode the audio stream to AAC format (optimized for playback devices)
                '-strict', 'experimental',  # Enable experimental features (needed for AAC encoding in some versions)
                subfolder_path         # Specify the output file path (video with audio)
            ]
            
            # Run the command 
            print('Now combining this Audio and Video')
            try:
                result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
                #print(result.stdout)
                
                print(f'Video with audio saved as {subfolder_path}')
                
            except subprocess.CalledProcessError as e:
                print(f"Error combining audio and video {video_path} and {audio_path}: {e.stderr}")
                    
            print('\n')
            
print(f'Done, you can now look into the folder: {output_audiovideo_folder}')


Processing the audio file:  f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs\pilot_01\day2\pilot_01_d2_speech_lsl_Mic_Speech_Start_SUD_1_Start_denoised.wav
Participant number: pilot_01
Day number: d2
Found matching video: f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\video_clipped\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start.avi for audio: f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs\pilot_01\day2\pilot_01_d2_speech_lsl_Mic_Speech_Start_SUD_1_Start_denoised.wav
Now combining this Audio and Video
Video with audio saved as f:\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\audiovideo_merged\marker_3pairs\pilot_01\d2\pilot_01_d2_Speech_Start_SUD_1_Start_denoised_audiovideo_merged.avi


Processing the audio file:  f:\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed_final\marker_3pairs\pilot_01\day3\pilot_01_d3_speech_lsl_Mic_Speech_Start_SUD_1_Start_denoised.wav
Participant number: pilot

## 4. (Optional) Overlay LSL Time on the AudioVideos 
##### This step is useful to double check whether the Marker & Event informaiton was coded correctly during the recording. 

In [6]:
## 1.  AGAIN, FIND ALL THE AUDIO-VIDEO SYNCED FILES IN THE INPUT FOLDER 
input_audiovideo_folder = './audiovideo_sync/lateststart_earliestend'     # this folder contains the audio-video synced files

audiovideo_list = []  # Initialize an empty list to store paths of video files (csv LSL data)
try:
    if os.path.exists(input_audiovideo_folder):  # Check if the input folder exists
       
        # Traverse through the directory and its subdirectories to find XDF files
        for root, dirs, files in os.walk(os.path.abspath(input_audiovideo_folder)):  # Walk through the directory tree
          
            for file in files:
                
                if "synced" in file and file.endswith(".avi"):  # Check that the words "speech" and "output_compr" are in the file name and that it is a avi file
                    
                    audiovideo_list.append(os.path.join(root, file))  # Append the full path to the xdf_files list
                    
        if audiovideo_list :  # Check if any files were found
            print(f'We have identified the following audiovideo files: {audiovideo_list}')
        
        else:  # If no files were found
            print('No video files found.')
            
    else: # If the input folder does not exist or is not accessible
        print(f'The folder {input_audiovideo_folder} does not exist or is not accessible.')

except Exception as e:
    print(f"An error occurred: {str(e)}")


## 2. FIND THE CORRESPONDING LSL VIDEO FILES IN THE INPUT FOLDER
# We use the LSL_video_list that was created before
print(f'We will use the following LSL Video Files to overlay LSL Times: {LSL_video_list}')


We have identified the following audiovideo files: ['c:\\Users\\ahmar\\OneDrive\\Documents\\GitHub\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\audiovideo_sync\\lateststart_earliestend\\pilot_01\\d2\\pilot_01_d2_speech_audiovideo_synced.avi', 'c:\\Users\\ahmar\\OneDrive\\Documents\\GitHub\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\audiovideo_sync\\lateststart_earliestend\\pilot_01\\d3\\pilot_01_d3_speech_audiovideo_synced.avi', 'c:\\Users\\ahmar\\OneDrive\\Documents\\GitHub\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\audiovideo_sync\\lateststart_earliestend\\pilot_02\\d2\\pilot_02_d2_speech_audiovideo_synced.avi', 'c:\\Users\\ahmar\\OneDrive\\Documents\\GitHub\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\audiovideo_sync\\lateststart_earliestend\\pilot_02\\d3\\pilot_02_d3_speech_audiovideo_synced.avi', 'c:\\Users\\ahmar\\OneDrive\\Documents\\GitHub\\SpeakUp-2.0\\2_PREPROCESSING\\2_Audio_Video_Sync\\audiovideo_sync\\lateststart_earliestend\\pilot_03\\d2\\pilot_

In [13]:
## 3 NOW WE OVERLAY LSL TIMES ON THE AUDIO-VIDEO SYNCED FILES


# 1. Loading the relevant CSV files for each participant
for file in LSL_video_list: 
    
    fnam = os.path.basename(file)[:-4]  # Extract the file name from the path and remove the '.csv' extension (i.e., the last 4 characters in the string)
    
    print(f'Processing the LSL video file {file}')
    
    file_path = os.path.join(os.path.abspath(input_file_folder), file)
    
    participant_num = '_'.join(fnam.split('_')[0:2])  # Extract the participant number from the file name by splitting the string at the underscores and selecting the first two elements
    
    day_num = fnam.split('_')[2]  # Extract the day number from the file name by splitting the string at the underscores and selecting the third element
    
    # Reading the CSV file 
    file_data = pd.read_csv(file) # Reads the CSV file at the constructed path into a DataFrame called file_data 

    # Extract LSL time and frame number from CSV
    lsl_times = file_data.iloc[:, 0]  
    lsl_frames = file_data.iloc[:, 1]  
    
    # Extract LSL video identifier 
    LSL_video_match = audio_video_pattern_matching.search(fnam)
    if not LSL_video_match: 
        continue 
    LSL_video_identifier = LSL_video_match.group(0) 
    
        
    
    # 2. Loading the corresponding audiovideo files 
    
    for audiovideo in audiovideo_list: 
        
        fnam = os.path.basename(audiovideo)[:-4]  # Extract the file name from the path and remove the '.avi' extension (i.e., the last 4 characters in the string)
        
        # Extract the audiovideo identifier based on the pattern: 
        audiovideo_match = audio_video_pattern_matching.search(fnam)
       #print(f'Video identifier: {video_identifier}')  # Print the matched identifier for video

        
        # Check tha that the LSL and the audio video identifier match 
        
        if audiovideo_identifier == LSL_video_identifier: 
            
            audiovideo_path = os.path.join(os.path.abspath(input_audiovideo_folder), audiovideo)
            
            print("Found matching audiovideo file: " + os.path.basename(audiovideo) + "for LSL video " + os.path.basename(file)) 
            
            
            # Now Loading the video and extracting relevant metadata 
            print('Now loading video: ' + audiovideo)
            capture = cv2.VideoCapture(audiovideo_path)  # Load the video using OpenCV
            
            video_frame_width  = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))  
            video_frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))  
            video_frame_rate   = capture.get(cv2.CAP_PROP_FPS)
            video_tot_frames   = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
               
            
            # Find the codec based on the video extension
            video_extension = os.path.splitext(audiovideo_path)[1].lower()  # Extract the file extension from the video file path, ensuring it's in lowercase (e.g., '.mp4'), to enable case-insensitive matching in the dictionary
            if video_extension in extension_to_codec: 
                codec = extension_to_codec[video_extension]
            else:
                raise ValueError(f"ERROR: The video extension {video_extension} is not supported")
            
            # Create the Output Folder for the overlayed videos
            subfolder_path = os.path.join(os.path.abspath(output_overlay_folder), participant_num, day_num, f'{participant_num}_{day_num}_Speech_Video_overlayed{video_extension}')  # Create the output subfolder for the clipped video with the correct extension
            # Make sure the output folder exists, if not create it
            if not os.path.exists(os.path.dirname(subfolder_path)):
                os.makedirs(os.path.dirname(subfolder_path), exist_ok=True)
            
            # Create the output video writer
            fourcc = cv2.VideoWriter_fourcc(*codec)  
            
            video_writer = cv2.VideoWriter(subfolder_path, fourcc, video_frame_rate, (video_frame_width, video_frame_height))
            
            capture = cv2.VideoCapture(audiovideo_path)  
            
            print('Now overlaying LSL time onto video frames for video ' + str(os.path.basename(audiovideo)) + " with csv file  " + str(os.path.basename(file)))
            

            # Process the video frame by frame
            for i in tqdm(range(video_tot_frames), desc="Processing Video", unit="frame"):
                ret, frame = capture.read()  # Read a frame
                if not ret:
                    break  # Exit loop if no more frames are available
                
                # Get the actual frame number from the video capture object
                current_frame_number = capture.get(cv2.CAP_PROP_POS_FRAMES)
                print("Current frame number: " + str(current_frame_number))

                # Check if the frame counter is within the range of LSL frames
                if lsl_frames.iloc[0] <= frame_counter <= lsl_frames.iloc[-1]:
                    # Find the corresponding LSL time and frame number based on the frame counter
                    match = file_data[file_data.iloc[:, 1] == int(current_frame_number)]
                    if not match.empty:
                        lsl_time = match.iloc[0, 0]  # LSL time
                        lsl_frame = match.iloc[0, 1]  # Frame number

                        # Overlay the LSL information onto the video frame
                        overlay_text = f'LSL Time: {lsl_time:.3f}, Frame: {int(lsl_frame)}'
                        # Calculate the center position for the text
                        text_size = cv2.getTextSize(overlay_text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
                        text_x = (video_frame_width - text_size[0]) // 2  # X-coordinate
                        text_y = (video_frame_height + text_size[1]) // 2  # Y-coordinate
                        cv2.putText(frame, overlay_text, 
                                    (text_x, text_y),  # Position in the center
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, 
                                    (255, 255, 255), 2)  # White text with thickness of 2

                # Write the frame to the output video, regardless of overlay
                video_writer.write(frame)

            # Release resources after processing all frames
            capture.release()
            video_writer.release()

            print(f'Video saved as {subfolder_path}')
            
            print("\n")
                        
            

print("Done with processing all videos! You can now look into your folder: " + output_overlay_folder)


     
                

Processing the LSL video file c:\Users\ahmar\OneDrive\Documents\GitHub\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed\lateststart_earliestend\pilot_01\day2\pilot_01_d2_speech_lsl_Video_latest_start_earliest_end.csv
Processing the LSL video file c:\Users\ahmar\OneDrive\Documents\GitHub\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed\lateststart_earliestend\pilot_01\day3\pilot_01_d3_speech_lsl_Video_latest_start_earliest_end.csv
Processing the LSL video file c:\Users\ahmar\OneDrive\Documents\GitHub\SpeakUp-2.0\2_PREPROCESSING\1_XDF_PROCESSING\data_processed\lateststart_earliestend\pilot_02\day2\pilot_02_d2_speech_lsl_Video_latest_start_earliest_end.csv
Found matching audiovideo file: pilot_01_d2_speech_audiovideo_synced.avifor LSL video pilot_02_d2_speech_lsl_Video_latest_start_earliest_end.csv
Now loading video: c:\Users\ahmar\OneDrive\Documents\GitHub\SpeakUp-2.0\2_PREPROCESSING\2_Audio_Video_Sync\audiovideo_sync\lateststart_earliestend\pilot_01\d2\pilot_01_d2_

Processing Video:   0%|          | 0/32120 [00:00<?, ?frame/s]

Current frame number: 1.0
Current frame number: 2.0
Current frame number: 3.0
Current frame number: 4.0
Current frame number: 5.0
Current frame number: 6.0
Current frame number: 7.0
Current frame number: 8.0
Current frame number: 9.0
Current frame number: 10.0
Current frame number: 11.0
Current frame number: 12.0
Current frame number: 13.0
Current frame number: 14.0
Current frame number: 15.0
Current frame number: 16.0
Current frame number: 17.0
Current frame number: 18.0
Current frame number: 19.0
Current frame number: 20.0
Current frame number: 21.0
Current frame number: 22.0
Current frame number: 23.0
Current frame number: 24.0
Current frame number: 25.0
Current frame number: 26.0
Current frame number: 27.0
Current frame number: 28.0
Current frame number: 29.0
Current frame number: 30.0
Current frame number: 31.0
Current frame number: 32.0
Current frame number: 33.0
Current frame number: 34.0
Current frame number: 35.0
Current frame number: 36.0
Current frame number: 37.0
Current fr

KeyboardInterrupt: 